In [11]:
from numpy import array, concatenate, zeros
from numpy.random import randn
from scipy.optimize import minimize
from openfermionpyscf import run_pyscf
from pyscf import ci
from pyscf import mp
from openfermion.config import *
from openfermionprojectq import *

from openfermion.hamiltonians import MolecularData
from openfermion.transforms import jordan_wigner
from openfermion.utils import uccsd_singlet_paramsize

from projectq.ops import X, All, Measure
from projectq.backends import CommandPrinter, CircuitDrawer

In [12]:
#menentukan molekul yang akan ditinjau
geometry = [["H", [0, 0, 0]],
            ["H", [0, 0, 0.74]]]
basis = "sto-3g"
multiplicity = 1
charge = 0
molecule = MolecularData(geometry, basis, multiplicity, charge)
h2_molecule = run_pyscf(molecule, run_mp2=True, run_cisd=True, run_ccsd=True, run_fci=True)

In [13]:
#Use a Jordan-Wigner encoding, and compress to remove 0 imaginary components
qubit_hamiltonian = jordan_wigner(molecule.get_molecular_hamiltonian())
qubit_hamiltonian.compress()
compiler_engine = uccsd_trotter_engine()

In [16]:
initial_amplitudes = [0, 0.05677]
wavefunction = compiler_engine.allocate_qureg(molecule.n_qubits)
evolution_operator = uccsd_singlet_evolution(initial_amplitudes, 
                                                 molecule.n_qubits, 
                                                 molecule.n_electrons)
print(evolution_operator)

exp(-1j * (0.0141925 Y0 Y1 Y2 X3 +
-0.0141925 Y0 X1 Y2 Y3 +
-0.0141925 X0 X1 Y2 X3 +
-0.0141925 X0 Y1 Y2 Y3 +
0.0141925 Y0 X1 X2 X3 +
0.0141925 Y0 Y1 X2 Y3 +
0.0141925 X0 Y1 X2 X3 +
-0.0141925 X0 X1 X2 Y3))


In [8]:
def energy_objective(packed_amplitudes):
    """Evaluate the energy of a UCCSD singlet wavefunction with packed_amplitudes
    Args:
        packed_amplitudes(ndarray): Compact array that stores the unique
            amplitudes for a UCCSD singlet wavefunction.
        
    Returns:
        energy(float): Energy corresponding to the given amplitudes
    """
    os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

    # Set Jordan-Wigner initial state with correct number of electrons
    wavefunction = compiler_engine.allocate_qureg(molecule.n_qubits)
    for i in range(molecule.n_electrons):
        X | wavefunction[i]

    # Build the circuit and act it on the wavefunction
    evolution_operator = uccsd_singlet_evolution(packed_amplitudes, 
                                                 molecule.n_qubits, 
                                                 molecule.n_electrons)
    evolution_operator | wavefunction
    compiler_engine.flush()

    # Evaluate the energy and reset wavefunction
    energy = compiler_engine.backend.get_expectation_value(qubit_hamiltonian, wavefunction)
    All(Measure) | wavefunction
    compiler_engine.flush()
    return energy



In [9]:
n_amplitudes = uccsd_singlet_paramsize(molecule.n_qubits, molecule.n_electrons)
initial_amplitudes = [0, 0.05677]
initial_energy = energy_objective(initial_amplitudes)

# Run VQE Optimization to find new CCSD parameters
opt_result = minimize(energy_objective, initial_amplitudes,
                      method="CG", options={'disp':True})

opt_energy, opt_amplitudes = opt_result.fun, opt_result.x
print("\nOptimal UCCSD Singlet Energy: {}".format(opt_energy))
print("Optimal UCCSD Singlet Amplitudes: {}".format(opt_amplitudes))
print("Classical CCSD Energy: {} Hartrees".format(molecule.ccsd_energy))
print("Exact FCI Energy: {} Hartrees".format(molecule.fci_energy))
print("Initial Energy of UCCSD with CCSD amplitudes: {} Hartrees".format(initial_energy))

Optimization terminated successfully.
         Current function value: -1.137284
         Iterations: 1
         Function evaluations: 12
         Gradient evaluations: 3

Optimal UCCSD Singlet Energy: -1.1372838344884901
Optimal UCCSD Singlet Amplitudes: [0.         0.05639139]
Classical CCSD Energy: -1.1372839986104415 Hartrees
Exact FCI Energy: -1.137283834488502 Hartrees
Initial Energy of UCCSD with CCSD amplitudes: -1.1372829054972706 Hartrees


In [10]:
compiler_engine = uccsd_trotter_engine(CommandPrinter())
wavefunction = compiler_engine.allocate_qureg(molecule.n_qubits)
for i in range(molecule.n_electrons):
    X | wavefunction[i]

# Build the circuit and act it on the wavefunction
evolution_operator = uccsd_singlet_evolution(opt_amplitudes, 
                                             molecule.n_qubits, 
                                             molecule.n_electrons)
evolution_operator | wavefunction
compiler_engine.flush()

Allocate | Qureg[0]
Allocate | Qureg[1]
Allocate | Qureg[2]
Allocate | Qureg[3]
X | Qureg[0]
X | Qureg[1]
Rx(1.570796326795) | Qureg[0]
Rx(1.570796326795) | Qureg[1]
Rx(1.570796326795) | Qureg[2]
H | Qureg[3]
CX | ( Qureg[0], Qureg[1] )
CX | ( Qureg[1], Qureg[2] )
CX | ( Qureg[2], Qureg[3] )
Rz(0.028195695108) | Qureg[3]
CX | ( Qureg[2], Qureg[3] )
CX | ( Qureg[1], Qureg[2] )
CX | ( Qureg[0], Qureg[1] )
H | Qureg[3]
Rx(10.995574287564) | Qureg[2]
Rx(10.995574287564) | Qureg[1]
Rx(10.995574287564) | Qureg[0]
Rx(1.570796326795) | Qureg[0]
H | Qureg[1]
Rx(1.570796326795) | Qureg[2]
Rx(1.570796326795) | Qureg[3]
CX | ( Qureg[0], Qureg[1] )
CX | ( Qureg[1], Qureg[2] )
CX | ( Qureg[2], Qureg[3] )
Rz(12.538174919251) | Qureg[3]
CX | ( Qureg[2], Qureg[3] )
CX | ( Qureg[1], Qureg[2] )
CX | ( Qureg[0], Qureg[1] )
Rx(10.995574287564) | Qureg[3]
Rx(10.995574287564) | Qureg[2]
H | Qureg[1]
Rx(10.995574287564) | Qureg[0]
H | Qureg[0]
H | Qureg[1]
Rx(1.570796326795) | Qureg[2]
H | Qureg[3]
CX | ( Qur